<a href="https://colab.research.google.com/github/anacasicande/Modelamiento/blob/main/tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sympy as sp
import numpy as np


P = 9.4573  # Presión en bar
T = 350.0   # Temperatura en K
R = 0.08314  # Constante de los gases en J/mol·K (convertir después a L·bar/mol·K)

# Propiedades críticas del n-butano
Tc = 425.2  # Temperatura crítica en K
Pc = 37.96  # Presión crítica en bar
tr = T / Tc  # Temperatura reducida
pr = P / Pc  # Presión reducida
Z = sp.Symbol('Z')  # Creo una variable simbolica para poder derivar usando la funcion diff

# Parámetros de la ecuación de Redlich-Kwong
q = (0.42748 / 0.08664) * (tr**(-3/2))  #  Formula de q que usa los valores
b = ((0.08664 * pr )/ tr)

# Función para Z_vapor usando Redlich-Kwong
def Rk_vapor(Z):  # Defino Rk_vapor como una funcion que recibe de parametros una Z_vapor
    return Z - (1 + b - q * b * ((Z - b) / (Z * (Z + b))))  # Retorna la ecuacion de Rk_vapor

# Derivada de la función Z_vapor
def dRk_vapor(Z_value):  # Defino dRk_vapor como una funcion que recibe de parametros una variable simbolica
    derivadaV = sp.diff(Rk_vapor(Z), Z)  # Derivo usando la funcion sp.diff que recibe como parametros la funcion a derivar y una variable simbolica
    return derivadaV.subs(Z, Z_value).evalf()  #  Retorna la derivada evaluada en el valor "Z"

# Función para Z_liquid usando Redlich-Kwong
def Rk_liquid(Z):  # Defino Rk_liquid como una funcion que recibe de parametros una Z_liquid
    return Z - (b + Z * (Z + b) * ((1 + b - Z) / (q * b)))  # Retorna la ecuacion de Rk_liquid

# Derivada de la función Z_liquid
def dRk_liquid(Z_value):  # Defino dRk_liquid como una funcion que recibe de parametros una variable simbolica
    derivadaL = sp.diff(Rk_liquid(Z), Z)  # Derivo usando la funcion sp.diff que recibe como parametros la funcion a derivar y una variable simbolica
    return derivadaL.subs(Z, Z_value).evalf()  #  Retorna la derivada evaluada en el valor "Z"

# Método de Newton-Raphson
def newton_raphson(func, func_prime, Z, tol=1e-10, max_iter=200):
    x = Z  # igualo la variable "Z" a "X" para facilidad en el codigo
    for i in range(max_iter):  # realizo un bucle que recorre hasta el numero de iteraciones puestas
        fx = func(x)  #  igualo la variable "fx" al parametro "func()" evaluado en x para facilidad en el codigo
        fpx = func_prime(x)  #  igualo la variable "fpx" al parametro "func_prime()" evaluado en x para facilidad en el codigo

        if abs(fpx) < tol:  #  Si en una interacion el valor absoluto de la derivada es menor a la tolerancia para el ciclo
            print("La derivada es muy pequeña, no se puede continuar")
            return None

        # Fórmula de Newton-Raphson
        x_new = x - fx / fpx

        # Verificar la convergencia
        if abs(x_new - x) < tol:
            print(f"Converge en la iteración {i+1}")
            return x_new

        x = x_new  # Actualizar x para la siguiente iteración

    print("No converge después del número máximo de iteraciones")
    return None

# Estimaciones iniciales para los factores de compresibilidad Z
Z_guess_vapor = 1  # # El libro de Smith, Van Ness sugiere inicializar Z en 1 para vapor saturado.
Z_guess_liquid = 0.005  # Aproximación para el líquido

# Calcular Z usando Newton-Raphson
Z_vapor = newton_raphson(Rk_vapor, dRk_vapor, Z_guess_vapor)
Z_liquid = newton_raphson(Rk_liquid, dRk_liquid, Z_guess_liquid)

# Convertir Z a volúmenes molares Vm (en L/mol) y luego a cm^3/mol
Vm_vapor_cm3 = Z_vapor * R * T / P *1000 if Z_vapor else None
Vm_liquid_cm3 = Z_liquid * R * T / P * 1000  if Z_liquid else None

Z_vapor, Z_liquid, Vm_vapor_cm3, Vm_liquid_cm3



Converge en la iteración 4
Converge en la iteración 6


(0.830353561570580, 0.0433061372400762, 2554.90026626440, 133.247891845345)